# Batch pixel count code for bone tissue mapping and creation of new variables 

The following code counts the number of pixels in a cortical bone tissue mapped image, creates new variables based on the pixel counts, and is meant for research purposes. However, the code can be modified to count any color in an image, provided that the color codes are changed.

This code can also be applied to batches of smaller-size image files (e.g., infants, juveniles, small monkeys, etc.), or to batches of larger images on a system with powerful computing power. 

**All code produces long format data frame.**

### Packages
Packages required 

In [ ]:
# !python3 -mpip install Pillow

### Modules
Modules to import/load

In [ ]:
from PIL import Image
import numpy as np
import pandas as pd

### Load Image

In [ ]:
# List of image filenames
image_files = ['image1.png', 'image2.png', 'image3.png']  # Replace with your filenames or paths

### Defining the colors and labels

In [ ]:
# Define the colors and labels
colors = {
    "WOV": "#0d2d1d",
    "FLC": "#12422d",
    "FLC-LZPO": "#135b42",
    "PF": "#1f7b5c",
    "PF-LAM": "#289871",
    "LAM": "#6bc3a6",
    "ELAM": "#e18174",
    "ECCC": "#e19a22",
    "EPF": "#b54f52",
    "EPF-LAM": "#dd615e",
    "SF": "#f6f4c4",
    "INT": "#b5e2b8",
    "HAV": "#4e5c91",
    "BK": "#ffffff" # Background color of white
}

### Convert hex to RGB

In [ ]:
# Function to convert hex to RGB
def hex_to_rgb(value):
    value = value.lstrip('#') 
    length = len(value) 
    return tuple(int(value[i:i+length//3], 16) for i in range(0, length, length//3)) 

In [ ]:
### Pixel count and new variables 

In [ ]:
# Function to count pixels for a given image
def process_image(image_path):
    img = Image.open(image_path)
    img_array = np.array(img)
    
    # Count pixels for each color
    pixel_counts = {} # Creates an empty dictionary 
    for label, color in colors.items():
        rgb_color = hex_to_rgb(color)
        count = np.sum(np.all(img_array[:, :, :3] == rgb_color, axis=2))
        pixel_counts[label] = count
        
    # Convert pixel_counts dictionary to a DataFrame
    df_pixel_counts = pd.DataFrame(list(pixel_counts.items()), columns=['Tissue Type', 'Pixel Count'])
    
    # Calculate C.Ar
    df_pixel_counts['C.Ar'] = df_pixel_counts['Pixel Count'].sum() - df_pixel_counts[df_pixel_counts['Tissue Type'] == 'BK']['Pixel Count'].iloc[0]
    
    # Calculate the percent of each tissue type based on C.Ar (excluding BK)
    c_ar_value = df_pixel_counts[df_pixel_counts['Tissue Type'] == 'C.Ar']['Pixel Count'].values[0]
    df_pixel_counts['Percent of C.Ar'] = np.where(df_pixel_counts['Tissue Type'] != 'BK', 
                                                       (df_pixel_counts['Pixel Count'] / c_ar_value) * 100, 
                                                       np.nan)
    
    # Calculate the sum for the tissues from WOV to LAM
    sum_tissues = df_pixel_counts[df_pixel_counts['Tissue Type'].isin(['WOV', 'FLC', 'FLC-LZPO', 'PF', 'PF-LAM', 'LAM'])]['Pixel Count'].sum()

    # Calculate the percent of each tissue type based on the sum
    df_pixel_counts['Percent of P.Ar'] = np.where(df_pixel_counts['Tissue Type'].isin(['WOV', 'FLC', 'FLC-LZPO', 'PF', 'PF-LAM', 'LAM']), 
                                                       (df_pixel_counts['Pixel Count'] / sum_tissues) * 100, 
                                                       np.nan)
    
    # Calculate the sum for the tissues from ELAM to EPF-LAM
    sum_tissues_e = df_pixel_counts[df_pixel_counts['Tissue Type'].isin(['ELAM', 'ECCC', 'EPF', 'EPF-LAM'])]['Pixel Count'].sum()

    # Calculate the percent of each tissue type based on the sum
    df_pixel_counts['Percent of E.Ar'] = np.where(df_pixel_counts['Tissue Type'].isin(['ELAM', 'ECCC', 'EPF', 'EPF-LAM']), 
                                                       (df_pixel_counts['Pixel Count'] / sum_tissues_e) * 100, 
                                                       np.nan)

    # Add a column for the image filename 
    df_pixel_counts['Image File'] = image_path
    
    return df_pixel_counts

### Data frame for results

In [ ]:
# List to hold DataFrames for each image
dfs = []

# Iterate over image files and process each one
for image_file in image_files:
    df_pixel_counts = process_image(image_file)
    dfs.append(df_pixel_counts)

# Combine all the DataFrames into a single DataFrame (if needed)
combined_df = pd.concat(dfs, axis=0).reset_index(drop=True)